# Master Discogs Database

In [143]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils, allmusicUtils
from collection import collections
from artist import artist
from searchUtils import findExt, findSubExt, findPatternExt
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from artist import artist
from artistAM import artistAM
from artistsAM import artistsAM
from albums import albums
from album import album, albumURLInfo
from time import sleep

from masterdb import discConv, isKnownAlbum, directoryName
from masterdb import getSlimArtistDB, getArtistNameToIDMap
from masterdb import getMusicData, getRowByIndex

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-01-31 21:17:07.416476


In [53]:
%load_ext autoreload
%autoreload
discDC = discogs()
arts = artists(discDC)
art  = artist()
albs = albums(disc)
alb  = album()
dutils = discogsUtils()

discAM   = discogs('allmusic')
artsAM   = artistsAM(discAM)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Warning! Saved Discog Directory None is not Available
Warning! Saved Discog Directory None is not Available


# Create Discogs <-> AllMusic Map

In [149]:
discSlimArtistDCDB = getSlimArtistDB(discDC, force=False)
artistDCNameToID   = getArtistNameToIDMap(discSlimArtistDCDB)
artistDCKeys       = artistDCNameToID.keys()

Current Time is Fri Jan 31, 2020 21:20:08 for 
=================================== Creating Artist DB ===================================
Using previously created Slim Artist DB
Current Time is Fri Jan 31, 2020 21:20:08 for Done with 
=================================== Creating Artist DB ===================================
Process [{0}] took 110.0 millseconds.


In [169]:
discSlimArtistAMDB = getSlimArtistDB(discAM, force=False)
artistAMNameToID   = getArtistNameToIDMap(discSlimArtistAMDB)
artistAMKeys       = artistAMNameToID.keys()

Current Time is Sun Feb 02, 2020 11:20:28 for 
=================================== Creating Artist DB ===================================
Using previously created Slim Artist DB
Current Time is Sun Feb 02, 2020 11:20:28 for Done with 
=================================== Creating Artist DB ===================================
Process [{0}] took 178.0 millseconds.
Current Time is Sun Feb 02, 2020 11:20:28 for 
=================================== Creating Artist Name To ID Map ===================================
Found 36432 ID -> Name entries
Found 34373 Name -> ID entries


In [170]:
artistAMToAlbumsData = discAM.getArtistIDToAlbumNamesData(debug=True)
print(len(artistAMToAlbumsData))

Data stored in /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToAlbumNames.p
Returning data from /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToAlbumNames.p
Loading data from /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToAlbumNames.p
  --> This file is 11.8MB.
Loading /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToAlbumNames.p
36432


In [59]:
artistDCToAlbumsData = discDC.getArtistIDToAlbumNamesData(debug=True)
print(len(artistDCToAlbumsData))

Data stored in /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToAlbumNames.p
Returning data from /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToAlbumNames.p
Loading data from /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToAlbumNames.p
  --> This file is 3.4MB.
Loading /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToAlbumNames.p
7559
Data stored in /Users/tgadfort/Music/Discog/db-discogs/ArtistIDToAlbumNames.p
Returning data from /Users/tgadfort/Music/Discog/db-discogs/ArtistIDToAlbumNames.p
Loading data from /Users/tgadfort/Music/Discog/db-discogs/ArtistIDToAlbumNames.p
  --> This file is 190.8MB.
Loading /Users/tgadfort/Music/Discog/db-discogs/ArtistIDToAlbumNames.p
712629


In [161]:
from searchUtils import findNearest
num    = 3
cutoff = 0.85

Albums
0000759680 I Wanna Be a Wonderful Milky


In [85]:
flat_list = [item for sublist in l for item in sublist]

[['I Wanna Be a Wonderful Milky']]

In [114]:
x = ['a', 'b', 'cc']
y = 'c'
y in x

False

In [171]:
artistAMIDtoDCIDMap = getFile(ifile="artistAMIDtoDCIDMap.p")
artistAMNametoIDMap = {v["AM Name"]: k for k,v in artistAMIDtoDCIDMap.items()}

In [172]:
#artistAMNameToDCNamesMap = {}

print(len(artistAMNameToDCNamesMap))

print(len(artistAMKeys))
for iKey,artistAMName in enumerate(artistAMKeys):
    if artistAMNametoIDMap.get(artistAMName):
        continue
    if artistAMName in artistDCKeys:
        artistAMNameToDCNamesMap[artistAMName] = [artistAMName]
        
    continue
    if True:
        artistDCNames = findNearest(artistAMName, artistDCKeys, num=3, cutoff=cutoff)
        if len(artistDCNames) > 0:
            print(iKey,'/',len(artistAMKeys),'\t',artistAMName)
            artistAMNameToDCNamesMap[artistAMName] = artistDCNames
            
print(len(artistAMNameToDCNamesMap))

12560
34373
18313


In [ ]:
from difflib import SequenceMatcher
import operator

def getArtistAlbums(vals):
    albums = []
    for k,v in vals.items():
        for k2, v2 in v.items():
            albums.append(v2)
    return albums
            
    
def getBestAlbumsMatch(albumsAM, albumsDC, debug=False):
    score = 0.0
    for albumAMAlbum in albumsAM:
        for albumDCAlbum in albumsDC:
            s = SequenceMatcher(None, albumAMAlbum, albumDCAlbum)
            ratio = round(s.ratio(), 3)
            if debug:
                print("\t\t\t[{0}] + [{1}] = [{2}]".format(albumAMAlbum, albumDCAlbum, ratio))
            if ratio > 0.7:
                score += ratio
    return score

def showNames(artistAMName, artistDCNames):
    print("{0: <50}{1}".format(artistAMName, artistDCNames))



artistAMIDtoDCIDMap = getFile(ifile="artistAMIDtoDCIDMap.p")
artistAMNametoIDMap = {v["AM Name"]: k for k,v in artistAMIDtoDCIDMap.items()}

for iKey,artistAMName in enumerate(artistAMKeys):
    
    if artistAMNametoIDMap.get(artistAMName):
        continue
    artistDCNames = artistAMNameToDCNamesMap.get(artistAMName)
    if artistDCNames is None:
        continue
    #artistDCNames = findNearest(artistAMName, artistDCKeys, num=3, cutoff=cutoff)
    #if len(artistDCNames) == 0:
    #    continue
    
    artistAMIDs = artistAMNameToID[artistAMName]
    
    ## Loop over Artist AM IDs
    for iAMID,artistAMID in enumerate(artistAMIDs):
        if artistAMIDtoDCIDMap.get(artistAMID) is not None:
            continue
        
        showNames(artistAMName, artistDCNames)
        artistAMAlbums = getArtistAlbums(artistAMToAlbumsData.get(artistAMID))
        if artistAMAlbums is None:
            print("\tAM ID={0: <15}{1: <5}/{2: <8}{3: <5}{4}".format(artistAMID, iKey, len(artistAMKeys), iAMID, "NONE"))
        else:
            print("\tAM ID={0: <15}{1: <5}/{2: <8}{3: <5}{4}".format(artistAMID, iKey, len(artistAMKeys), iAMID, len(artistAMAlbums)))

            
        artistAMtoDCScore = None
        
            
        ## Loop over Artist DC Names
        for artistDCName in artistDCNames:
            artistDCIDs = artistDCNameToID[artistDCName]
            
            ## Loop over Artist DC IDs
            for artistDCID in artistDCIDs:
                artistDCAlbums = getArtistAlbums(artistDCToAlbumsData.get(artistDCID))
                if False:
                    if artistDCAlbums is None:
                        print("\t\tDC ID={0: <15}\t      {1}".format(artistDCID, "NONE"))
                    else:
                        print("\t\tDC ID={0: <15}\t Size={1}".format(artistDCID, len(artistDCAlbums)))
                
                retval = getBestAlbumsMatch(artistAMAlbums, artistDCAlbums, debug=False)
                if retval > 0.7:
                    if artistAMtoDCScore is None:
                        artistAMtoDCScore = {"Name": artistDCName, "ID": artistDCID, "Score": retval}
                    else:
                        if retval > artistAMtoDCScore["Score"]:
                            artistAMtoDCScore = {"Name": artistDCName, "ID": artistDCID, "Score": retval}
                            
                            
        print("\tAM ID={0: <15}{1}".format(artistAMID, artistAMtoDCScore))
        if artistAMtoDCScore is not None:
            artistAMIDtoDCIDMap[artistAMID] = {"AM Name": artistAMName, "DC ID": artistAMtoDCScore["ID"], "DC Name": artistAMtoDCScore["Name"]}
            print("Found Match for {0} --> {1}".format(artistAMID, artistAMIDtoDCIDMap[artistAMID]))
            saveFile(idata=artistAMIDtoDCIDMap, ifile="artistAMIDtoDCIDMap.p")
            print("\n")
        


Garnet Mimms                                      ['Garnet Mimms']
	AM ID=0000965206     1    /34373   0    16
	AM ID=0000965206     {'Name': 'Garnet Mimms', 'ID': '363202', 'Score': 16.081}
Found Match for 0000965206 --> {'AM Name': 'Garnet Mimms', 'DC ID': '363202', 'DC Name': 'Garnet Mimms'}
  --> This file is 499.3kB.


The Wags                                          ['The Wags']
	AM ID=0001379506     2    /34373   0    2
	AM ID=0001379506     {'Name': 'The Wags', 'ID': '1630979', 'Score': 0.957}
Found Match for 0001379506 --> {'AM Name': 'The Wags', 'DC ID': '1630979', 'DC Name': 'The Wags'}
  --> This file is 499.3kB.


Bebe Rebozo                                       ['Bebe Rebozo']
	AM ID=0001617506     9    /34373   0    1
	AM ID=0001617506     None
Lauren Wood                                       ['Lauren Wood']
	AM ID=0000142106     10   /34373   0    7
	AM ID=0000142106     {'Name': 'Lauren Wood', 'ID': '270134', 'Score': 5.036}
Found Match for 0000142106 --> {'AM Name'

	AM ID=0000763506     {'Name': 'Derek Bailey', 'ID': '41904', 'Score': 81.01700000000001}
Found Match for 0000763506 --> {'AM Name': 'Derek Bailey', 'DC ID': '41904', 'DC Name': 'Derek Bailey'}
  --> This file is 499.9kB.


D.Y.S.                                            ['D.I.Y.S.']
	AM ID=0000554406     107  /34373   0    4
	AM ID=0000554406     None
Shelf Life                                        ['Shelf Life']
	AM ID=0003420206     111  /34373   0    1
	AM ID=0003420206     None
Lynx                                              ['Lynx']
	AM ID=0001466306     113  /34373   0    1
	AM ID=0001466306     {'Name': 'Lynx', 'ID': '696627', 'Score': 2.0}
Found Match for 0001466306 --> {'AM Name': 'Lynx', 'DC ID': '696627', 'DC Name': 'Lynx'}
  --> This file is 500.0kB.


Lynx                                              ['Lynx']
	AM ID=0001003103     113  /34373   1    13
	AM ID=0001003103     {'Name': 'Lynx', 'ID': '1813127', 'Score': 2.941}
Found Match for 0001003103 --> {'AM Name': 

  --> This file is 500.5kB.


Dominion                                          ['Dominion']
	AM ID=0000558084     164  /34373   4    1
	AM ID=0000558084     {'Name': 'Dominion', 'ID': '1524299', 'Score': 0.944}
Found Match for 0000558084 --> {'AM Name': 'Dominion', 'DC ID': '1524299', 'DC Name': 'Dominion'}
  --> This file is 500.5kB.


Cherubia                                          ['Cherub']
	AM ID=0001873906     165  /34373   0    1
	AM ID=0001873906     None
Rains                                             ['Rains']
	AM ID=0002067506     167  /34373   0    2
	AM ID=0002067506     None
Vampire Beach Babes                               ['Vampire Beach Babes']
	AM ID=0000518406     175  /34373   0    2
	AM ID=0000518406     {'Name': 'Vampire Beach Babes', 'ID': '786294', 'Score': 1.976}
Found Match for 0000518406 --> {'AM Name': 'Vampire Beach Babes', 'DC ID': '786294', 'DC Name': 'Vampire Beach Babes'}
  --> This file is 500.6kB.


Monsters At Work                               

	AM ID=0000212006     {'Name': 'Ricochet', 'ID': '1576249', 'Score': 9.643}
Found Match for 0000212006 --> {'AM Name': 'Ricochet', 'DC ID': '1576249', 'DC Name': 'Ricochet'}
  --> This file is 501.1kB.


Ricochet                                          ['Ricochet']
	AM ID=0000471215     280  /34373   1    2
	AM ID=0000471215     None
Ricochet                                          ['Ricochet']
	AM ID=0000908293     280  /34373   2    2
	AM ID=0000908293     {'Name': 'Ricochet', 'ID': '729170', 'Score': 0.936}
Found Match for 0000908293 --> {'AM Name': 'Ricochet', 'DC ID': '729170', 'DC Name': 'Ricochet'}
  --> This file is 501.1kB.


Ricochet                                          ['Ricochet']
	AM ID=0003254832     280  /34373   3    1
	AM ID=0003254832     None
Ricochet                                          ['Ricochet']
	AM ID=0001399671     280  /34373   4    1
	AM ID=0001399671     None
The Underdog                                      ['The Underdog']
	AM ID=0001285906     

  --> This file is 501.7kB.


The Indecision                                    ['The Indecision']
	AM ID=0003535194     377  /34373   0    2
	AM ID=0003535194     {'Name': 'The Indecision', 'ID': '3483665', 'Score': 1.971}
Found Match for 0003535194 --> {'AM Name': 'The Indecision', 'DC ID': '3483665', 'DC Name': 'The Indecision'}
  --> This file is 501.7kB.


Todd Reynolds                                     ['Todd Reynolds']
	AM ID=0000206794     381  /34373   0    4
	AM ID=0000206794     {'Name': 'Todd Reynolds', 'ID': '507615', 'Score': 2.0}
Found Match for 0000206794 --> {'AM Name': 'Todd Reynolds', 'DC ID': '507615', 'DC Name': 'Todd Reynolds'}
  --> This file is 501.8kB.


Halo                                              ['Halo']
	AM ID=0003608894     387  /34373   0    2
	AM ID=0003608894     None
Halo                                              ['Halo']
	AM ID=0003598899     387  /34373   1    1
	AM ID=0003598899     None
Halo                                              ['

...

  --> This file is 610.8kB.


Rheinische Kantorei                               ['Rheinische Kantorei']
	AM ID=0001924190     19693/34373   0    29
	AM ID=0001924190     None
Blacc Zacc                                        ['Blacc Zacc']
	AM ID=0003867590     19697/34373   0    3
	AM ID=0003867590     None
New Fumes                                         ['New Fumes']
	AM ID=0002955290     19701/34373   0    1
	AM ID=0002955290     None
Cecil Sharp                                       ['Cecil Sharp']
	AM ID=0000682390     19703/34373   0    1
	AM ID=0000682390     None
Johnnie Foreigner                                 ['Johnny Foreigner']
	AM ID=0001423890     19707/34373   0    1
	AM ID=0001423890     None
Diana Ross & the Supremes                         ['Diana Ross And The Supremes*']
	AM ID=0000821590     19711/34373   0    77
	AM ID=0000821590     None
Trey Parker                                       ['Trey Parker']
	AM ID=0000748190     19715/34373   0    2
	AM ID=000074819

  --> This file is 611.3kB.


Eddie C. Campbell                                 ['Eddie C. Campbell']
	AM ID=0000169690     19816/34373   0    9
	AM ID=0000169690     {'Name': 'Eddie C. Campbell', 'ID': '1634835', 'Score': 9.536999999999999}
Found Match for 0000169690 --> {'AM Name': 'Eddie C. Campbell', 'DC ID': '1634835', 'DC Name': 'Eddie C. Campbell'}
  --> This file is 611.3kB.


Cinerama                                          ['Cinerama']
	AM ID=0000127990     19820/34373   0    26
	AM ID=0000127990     {'Name': 'Cinerama', 'ID': '137762', 'Score': 25.911}
Found Match for 0000127990 --> {'AM Name': 'Cinerama', 'DC ID': '137762', 'DC Name': 'Cinerama'}
  --> This file is 611.4kB.


Cinerama                                          ['Cinerama']
	AM ID=0002992846     19820/34373   1    1
	AM ID=0002992846     None
Michael McDaeth                                   ['Michael Meth']
	AM ID=0000871090     19822/34373   0    10
	AM ID=0000871090     None
Anti-Pasti                     

  --> This file is 612.0kB.


Tiger Lou                                         ['Tiger Lou']
	AM ID=0001640990     19943/34373   0    8
	AM ID=0001640990     {'Name': 'Tiger Lou', 'ID': '364859', 'Score': 8.629}
Found Match for 0001640990 --> {'AM Name': 'Tiger Lou', 'DC ID': '364859', 'DC Name': 'Tiger Lou'}
  --> This file is 612.1kB.


The Love Sunrise Orchestra                        ['The Sunrise Orchestra', 'The Velvet Sunrise Orchestra']
	AM ID=0003039490     19944/34373   0    1
	AM ID=0003039490     None
The Surf Zombies                                  ['The Surf Zombies']
	AM ID=0001589490     19946/34373   0    5
	AM ID=0001589490     {'Name': 'The Surf Zombies', 'ID': '3699340', 'Score': 3.909}
Found Match for 0001589490 --> {'AM Name': 'The Surf Zombies', 'DC ID': '3699340', 'DC Name': 'The Surf Zombies'}
  --> This file is 612.1kB.


Doa                                               ['Doa']
	AM ID=0001947290     19954/34373   0    10
	AM ID=0001947290     None
Aqua Boog

  --> This file is 612.6kB.


Dark Avenger                                      ['Dark Avenger']
	AM ID=0000396702     20083/34373   0    4
	AM ID=0000396702     {'Name': 'Dark Avenger', 'ID': '2055965', 'Score': 3.69}
Found Match for 0000396702 --> {'AM Name': 'Dark Avenger', 'DC ID': '2055965', 'DC Name': 'Dark Avenger'}
  --> This file is 612.7kB.


Peste Noire                                       ['Peste Noire']
	AM ID=0001057002     20102/34373   0    3
	AM ID=0001057002     {'Name': 'Peste Noire', 'ID': '697872', 'Score': 1.818}
Found Match for 0001057002 --> {'AM Name': 'Peste Noire', 'DC ID': '697872', 'DC Name': 'Peste Noire'}
  --> This file is 612.7kB.


Robert Solomon                                    ['Robert Solomon']
	AM ID=0001540902     20106/34373   0    1
	AM ID=0001540902     None
Utah Saints                                       ['Utah Saints']
	AM ID=0000304102     20110/34373   0    12
	AM ID=0000304102     {'Name': 'Utah Saints', 'ID': '2703', 'Score': 14.4549

  --> This file is 613.3kB.


Endless Boogie                                    ['Endless Boogie']
	AM ID=0000996802     20195/34373   0    8
	AM ID=0000996802     {'Name': 'Endless Boogie', 'ID': '654261', 'Score': 3.916}
Found Match for 0000996802 --> {'AM Name': 'Endless Boogie', 'DC ID': '654261', 'DC Name': 'Endless Boogie'}
  --> This file is 613.3kB.


Lo Moon                                           ['Lo Moon']
	AM ID=0003601802     20198/34373   0    1
	AM ID=0003601802     {'Name': 'Lo Moon', 'ID': '5789450', 'Score': 1.0}
Found Match for 0003601802 --> {'AM Name': 'Lo Moon', 'DC ID': '5789450', 'DC Name': 'Lo Moon'}
  --> This file is 613.4kB.


Bitter End                                        ['Bitter End']
	AM ID=0000107102     20199/34373   0    4
	AM ID=0000107102     {'Name': 'Bitter End', 'ID': '1440566', 'Score': 2.803}
Found Match for 0000107102 --> {'AM Name': 'Bitter End', 'DC ID': '1440566', 'DC Name': 'Bitter End'}
  --> This file is 613.4kB.


Bitter End      

In [164]:
len(artistAMIDtoDCIDMap)

14199

# Create Dictionary Lookup Files

## Artist ID --> Ref and Name

#### Dictionaries

In [165]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistIDMap
createArtistIDMap(discAM)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Sun Feb 02, 2020 11:18:26 for Creating Artist DBs
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/6-Metadata.p  	0 368
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/94-Metadata.p  	1 730
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/77-Metadata.p  	2 1094
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/40-Metadata.p  	3 1465
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/15-Metadata.p  	4 1801
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/22-Metadata.p  	5 2144
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/88-Metadata.p  	6 2519
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/99-Metadata.p  	7 2862
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/33-Metadata.p  	8 3258
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/51-Metadata.p  	9 3639
/Users/tgadfort/Music/Discog/artists-al

  --> This file is 924.0kB.
Saved data to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToRef.p
  --> This file is 924.0kB.
Saving 36432 entries to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToName.p

Saving data to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToName.p
  --> This file is 713.8kB.
Saved data to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToName.p
  --> This file is 713.8kB.
Saving 36432 entries to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToVariations.p

Saving data to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToVariations.p
  --> This file is 826.5kB.
Saved data to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToVariations.p
  --> This file is 826.5kB.
Current Time is Sun Feb 02, 2020 11:18:30 for Done with Creating Artist DBs
Process [{0}] took 4 seconds.


In [22]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistIDMap
createArtistIDMap(disc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Sun Jan 26, 2020 19:26:56 for Creating Artist DBs
/Volumes/Music/Discog/artists-db/metadata/0-Metadata.p  	0 7042
/Volumes/Music/Discog/artists-db/metadata/1-Metadata.p  	1 14333
/Volumes/Music/Discog/artists-db/metadata/10-Metadata.p  	2 21410
/Volumes/Music/Discog/artists-db/metadata/11-Metadata.p  	3 28588
/Volumes/Music/Discog/artists-db/metadata/12-Metadata.p  	4 35774
/Volumes/Music/Discog/artists-db/metadata/13-Metadata.p  	5 43002
/Volumes/Music/Discog/artists-db/metadata/14-Metadata.p  	6 50003
/Volumes/Music/Discog/artists-db/metadata/15-Metadata.p  	7 57131
/Volumes/Music/Discog/artists-db/metadata/16-Metadata.p  	8 64198
/Volumes/Music/Discog/artists-db/metadata/17-Metadata.p  	9 71368
/Volumes/Music/Discog/artists-db/metadata/18-Metadata.p  	10 78533
/Volumes/Music/Discog/artists-db/metadata/19-Metadata.p  	11 85666
/Volumes/Music/Discog/artists-db/metadata/2-Metadata.p 

#### Pandas DB

In [166]:
from masterdb import getSlimArtistDB
discSlimArtistAMDB = getSlimArtistDB(discAM, force=True)
discSlimArtistAMDB.head()

Current Time is Sun Feb 02, 2020 11:18:35 for 
=================================== Creating Artist DB ===================================
Loading ArtistID Data
Creating Pandas DataFrame for 36432 Artists
	Shape --> (36432, 1)
  Finding Real Artist Name
	Shape --> (36432, 3)
  Removing None Artist
	Shape --> (36432, 3)
  Finding Disc Artist Name
	Shape --> (36432, 4)
DataFrame Shape is (36432, 4)
Current Time is Sun Feb 02, 2020 11:18:36 for Done with 
=================================== Creating Artist DB ===================================
Process [{0}] took 998.0 millseconds.
Saving Master Artist DB File
  --> This file is 2.8MB.


,Name,Artist,Num,DiscArtist
0000414806,Swell Mob,Swell Mob,None,Swell Mob
0000965206,Garnet Mimms,Garnet Mimms,None,Garnet Mimms
0001379506,The Wags,The Wags,None,The Wags
0002037706,Lakker,Lakker,None,Lakker
0000641506,Inspector Nine,Inspector Nine,None,Inspector Nine


In [26]:
from masterdb import getSlimArtistDB
discSlimArtistDB = getSlimArtistDB(disc, force=True)
discSlimArtistDB.head()

Current Time is Sun Jan 26, 2020 19:38:42 for 
=================================== Creating Artist DB ===================================
Loading ArtistID Data
Loading data from /Volumes/Music/Discog/db/ArtistIDToName.p
  --> This file is 13.6MB.
Loading /Volumes/Music/Discog/db/ArtistIDToName.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToRef.p
  --> This file is 16.2MB.
Loading /Volumes/Music/Discog/db/ArtistIDToRef.p
Creating Pandas DataFrame for 712629 Artists
	Shape --> (712629, 1)
  Finding Real Artist Name
	Shape --> (712629, 3)
  Removing None Artist
	Shape --> (712629, 3)
  Finding Disc Artist Name
	Shape --> (712629, 4)
DataFrame Shape is (712629, 4)
Current Time is Sun Jan 26, 2020 19:39:09 for Done with 
=================================== Creating Artist DB ===================================
Process [{0}] took 26 seconds.
Saving Master Artist DB File
  --> This file is 57.0MB.


,Name,Artist,Num,DiscArtist
1000,Dave Clarke,Dave Clarke,NaN,Dave Clarke
1000500,Club Pulse,Club Pulse,NaN,Club Pulse
100200,Dike,Dike,NaN,Dike
1002000,Larry Stokes,Larry Stokes,NaN,Larry Stokes
1002600,Gidd Sanchez,Gidd Sanchez,NaN,Gidd Sanchez


## Artist ID --> Albums

#### Dictionaries

In [167]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistAlbumIDMap
createArtistAlbumIDMap(discAM)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Sun Feb 02, 2020 11:18:55 for Creating Artist DBs
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/51-MediaMetadata.p 	381       381       1606      
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/6-MediaMetadata.p 	749       749       3244      
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/16-MediaMetadata.p 	1122      1122      4878      
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/32-MediaMetadata.p 	1502      1502      6478      
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/94-MediaMetadata.p 	1864      1864      8111      
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/75-MediaMetadata.p 	2244      2244      9934      
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/48-MediaMetadata.p 	2626      2626      11692     
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/47-MediaMetadata.p 	2975  

/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/96-MediaMetadata.p 	27301     27301     120551    
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/30-MediaMetadata.p 	27706     27706     122353    
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/77-MediaMetadata.p 	28070     28070     123920    
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/53-MediaMetadata.p 	28420     28420     125452    
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/14-MediaMetadata.p 	28759     28759     126942    
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/4-MediaMetadata.p 	29132     29132     128469    
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/29-MediaMetadata.p 	29492     29492     130035    
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/61-MediaMetadata.p 	29846     29846     131738    
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/26-MediaMetadata.p 	30185     30185     133290    
/Users/tgadfort/Musi

In [27]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistAlbumIDMap
createArtistAlbumIDMap(disc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Sun Jan 26, 2020 19:51:41 for Creating Artist DBs
/Volumes/Music/Discog/artists-db/metadata/0-MediaMetadata.p 	7042      3616      19379     
/Volumes/Music/Discog/artists-db/metadata/1-MediaMetadata.p 	14333     7313      39865     
/Volumes/Music/Discog/artists-db/metadata/10-MediaMetadata.p 	21410     10958     59764     
/Volumes/Music/Discog/artists-db/metadata/11-MediaMetadata.p 	28588     14632     79734     
/Volumes/Music/Discog/artists-db/metadata/12-MediaMetadata.p 	35774     18344     99718     
/Volumes/Music/Discog/artists-db/metadata/13-MediaMetadata.p 	43002     22014     119840    
/Volumes/Music/Discog/artists-db/metadata/14-MediaMetadata.p 	50003     25628     139333    
/Volumes/Music/Discog/artists-db/metadata/15-MediaMetadata.p 	57131     29263     159809    
/Volumes/Music/Discog/artists-db/metadata/16-MediaMetadata.p 	64198     32872     179952    
/Volumes/Mu

#### Pandas DB

In [168]:
from masterdb import getArtistAlbumsDB
discArtistAMAlbumsDB = getArtistAlbumsDB(discAM, force=True)
discArtistAMAlbumsDB.head()

Current Time is Sun Feb 02, 2020 11:19:59 for 
=================================== Creating Artist Albums DB ===================================
Loading ArtistID Data
Creating Pandas DataFrame for 36432 Artists
	Shape --> (36432, 1)
DataFrame Shape is (36432, 1)
Saving Master Artist Albums DB File
  --> This file is 23.2MB.
Current Time is Sun Feb 02, 2020 11:20:08 for Done with 
=================================== Creating Artist Albums DB ===================================
Process [{0}] took 8 seconds.


,Albums
0002335451,"{'Albums': {'0002044846': 'Lazer Sword', '0002..."
0003744751,{'Albums': {'0003180388': 'Stay Awake'}}
0003894751,{'Albums': {'0003343834': 'Jeanjeanie'}}
0000667151,"{'Albums': {}, 'Single/EP': {'0002301647': 'Ge..."
0000923051,"{'Albums': {'0000010634': 'The Word', '0002835..."


In [28]:
from masterdb import getArtistAlbumsDB
discArtistAlbumsDB = getArtistAlbumsDB(disc, force=True)
discArtistAlbumsDB.head()

Current Time is Sun Jan 26, 2020 20:09:16 for 
=================================== Creating Artist Albums DB ===================================
Loading ArtistID Data
Loading data from /Volumes/Music/Discog/db/ArtistIDToAlbumNames.p
  --> This file is 190.8MB.
Loading /Volumes/Music/Discog/db/ArtistIDToAlbumNames.p
Creating Pandas DataFrame for 712629 Artists
	Shape --> (712629, 1)
DataFrame Shape is (712629, 1)
Saving Master Artist Albums DB File
  --> This file is 375.0MB.
Current Time is Sun Jan 26, 2020 20:12:33 for Done with 
=================================== Creating Artist Albums DB ===================================
Process [{0}] took 3.3 minutes.


,Albums
1000,"{'Albums': {'44815': 'Archive One', '44831': '..."
1000500,{'Miscellaneous': {'1178875': 'Peak Controller'}}
100200,"{'Albums': {'50511': 'PottpÃ¼ree'}, 'Singles &..."
1002000,{'Instruments & Performance': {'1002108': 'Are...
1002600,{'Vocals': {'1180905': 'Quiero'}}


## Artist ID --> Genre, Style, Artists Lookup Table

In [13]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistMetadataMap
createArtistMetadataMap(discAM)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Wed Jan 29, 2020 18:25:26 for Creating Artist DBs



Saving 0 entries to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToGenre.p

Saving data to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToGenre.p
  --> This file is 14B.
Saved data to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToGenre.p
  --> This file is 14B.
Saving 0 entries to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToStyle.p

Saving data to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToStyle.p
  --> This file is 14B.
Saved data to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToStyle.p
  --> This file is 14B.
Saving 0 entries to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToCollaborations.p

Saving data to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToCollaborations.p
  --> This file is 14B.
Saved data to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToCollaborations.p
  --> This file

In [35]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistMetadataMap
createArtistMetadataMap(disc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Thu Jan 23, 2020 21:27:15 for Creating Artist DBs
/Volumes/Music/Discog/albums-db/metadata/0-ArtistMetadata.p 	4771
/Volumes/Music/Discog/albums-db/metadata/1-ArtistMetadata.p 	9481
/Volumes/Music/Discog/albums-db/metadata/10-ArtistMetadata.p 	16165
/Volumes/Music/Discog/albums-db/metadata/11-ArtistMetadata.p 	22936
/Volumes/Music/Discog/albums-db/metadata/12-ArtistMetadata.p 	29738
/Volumes/Music/Discog/albums-db/metadata/13-ArtistMetadata.p 	33792
/Volumes/Music/Discog/albums-db/metadata/14-ArtistMetadata.p 	40028
/Volumes/Music/Discog/albums-db/metadata/15-ArtistMetadata.p 	46750
/Volumes/Music/Discog/albums-db/metadata/16-ArtistMetadata.p 	52022
/Volumes/Music/Discog/albums-db/metadata/17-ArtistMetadata.p 	56703
/Volumes/Music/Discog/albums-db/metadata/18-ArtistMetadata.p 	63415
/Volumes/Music/Discog/albums-db/metadata/19-ArtistMetadata.p 	69777
/Volumes/Music/Discog/albums-db/me

## Album ID --> Name, Ref, Artists Lookup Table

In [36]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createAlbumIDMap
createAlbumIDMap(disc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Thu Jan 23, 2020 21:31:37 for Creating Artist DBs
/Volumes/Music/Discog/albums-db/metadata/0-ArtistAlbums.p 	21825
/Volumes/Music/Discog/albums-db/metadata/1-ArtistAlbums.p 	41453
/Volumes/Music/Discog/albums-db/metadata/10-ArtistAlbums.p 	67791
/Volumes/Music/Discog/albums-db/metadata/11-ArtistAlbums.p 	89756
/Volumes/Music/Discog/albums-db/metadata/12-ArtistAlbums.p 	116582
/Volumes/Music/Discog/albums-db/metadata/13-ArtistAlbums.p 	129077
/Volumes/Music/Discog/albums-db/metadata/14-ArtistAlbums.p 	152458
/Volumes/Music/Discog/albums-db/metadata/15-ArtistAlbums.p 	174344
/Volumes/Music/Discog/albums-db/metadata/16-ArtistAlbums.p 	192435
/Volumes/Music/Discog/albums-db/metadata/17-ArtistAlbums.p 	206069
/Volumes/Music/Discog/albums-db/metadata/18-ArtistAlbums.p 	230543
/Volumes/Music/Discog/albums-db/metadata/19-ArtistAlbums.p 	250792
/Volumes/Music/Discog/albums-db/metadata/2-Artis

# Master Lookup Tests

In [ ]:
%load_ext autoreload
%autoreload
from lookup import testLookupMaps
testLookupMaps(disc)

# Pandas DB

## Slim Artist DB

In [13]:
from masterdb import getSlimArtistDB
discSlimArtistDB = getSlimArtistDB(disc)
discSlimArtistDB.head()

Current Time is Sun Jan 26, 2020 18:43:38 for 
=================================== Creating Artist DB ===================================
Loading ArtistID Data
Loading data from /Volumes/Music/Discog/db/ArtistIDToName.p
  --> This file is 13.6MB.
Loading /Volumes/Music/Discog/db/ArtistIDToName.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToRef.p
  --> This file is 16.2MB.
Loading /Volumes/Music/Discog/db/ArtistIDToRef.p
Creating Pandas DataFrame for 712628 Artists
	Shape --> (712628, 1)
  Finding Real Artist Name
	Shape --> (712628, 3)
  Removing None Artist
	Shape --> (712628, 3)
  Finding Disc Artist Name
	Shape --> (712628, 4)
DataFrame Shape is (712628, 4)
Current Time is Sun Jan 26, 2020 18:43:57 for Done with 
=================================== Creating Artist DB ===================================
Process [{0}] took 19 seconds.
Saving Master Artist DB File
  --> This file is 57.0MB.


,Name,Artist,Num,DiscArtist
1000,Dave Clarke,Dave Clarke,NaN,Dave Clarke
1000500,Club Pulse,Club Pulse,NaN,Club Pulse
100200,Dike,Dike,NaN,Dike
1002000,Larry Stokes,Larry Stokes,NaN,Larry Stokes
1002600,Gidd Sanchez,Gidd Sanchez,NaN,Gidd Sanchez


## Artist DB

In [19]:
from masterdb import getArtistDB

In [20]:
discArtistDB = getArtistDB(disc)
discArtistDB.head()

Current Time is Wed Jan 22, 2020 20:51:53 for 
=================================== Creating Artist DB ===================================
Loading ArtistID Data
Loading data from /Volumes/Music/Discog/db/ArtistIDToName.p
  --> This file is 13.6MB.
Loading /Volumes/Music/Discog/db/ArtistIDToName.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToRef.p
  --> This file is 16.2MB.
Loading /Volumes/Music/Discog/db/ArtistIDToRef.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToVariations.p
  --> This file is 26.6MB.
Loading /Volumes/Music/Discog/db/ArtistIDToVariations.p
Creating Pandas DataFrame for 712336 Artists
	Shape --> (712336, 1)
  Joining Ref
	Shape --> (712336, 2)
  Joining Variations
	Shape --> (712336, 3)
  Finding Real Artist Name
	Shape --> (712336, 6)
DataFrame Shape is (712336, 6)
Current Time is Wed Jan 22, 2020 20:52:30 for Done with 
=================================== Creating Artist DB ===================================
Process [{0}] took 36 seconds.
Saving Mas

,Name,Ref,Variations,Known,Artist,Num
1000,Dave Clarke,/artist/1000-Dave-Clarke,"[Dave Clarke, 7 Red 7, Clarck, Clarke, Clarke,...",True,Dave Clarke,NaN
1000500,Club Pulse,/artist/1000500-Club-Pulse,[Club Pulse],True,Club Pulse,NaN
100200,Dike,/artist/100200-Dike,"[Dike, D.I.K.E., Dike D, Uchegdu]",True,Dike,NaN
1002000,Larry Stokes,/artist/1002000-Larry-Stokes,[Larry Stokes],True,Larry Stokes,NaN
1002600,Gidd Sanchez,/artist/1002600-Gidd-Sanchez,[Gidd Sanchez],True,Gidd Sanchez,NaN


## Artist Metadata DB

In [39]:
from masterdb import getArtistMetadataDB

In [40]:
discArtistMetadataDB = getArtistMetadataDB(disc)
discArtistMetadataDB.head()

Current Time is Fri Jan 24, 2020 07:20:08 for 
=================================== Creating Artist Metadata DB ===================================
Loading ArtistID Data
Loading data from /Volumes/Music/Discog/db/ArtistIDToGenre.p
  --> This file is 12.1MB.
Loading /Volumes/Music/Discog/db/ArtistIDToGenre.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToStyle.p
  --> This file is 17.4MB.
Loading /Volumes/Music/Discog/db/ArtistIDToStyle.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToCollaborations.p
  --> This file is 37.7MB.
Loading /Volumes/Music/Discog/db/ArtistIDToCollaborations.p
Creating Pandas DataFrame for 572424 Artists
	Shape --> (572424, 1)
  Joining Style
	Shape --> (572424, 2)
  Joining Collaboration
	Shape --> (572424, 3)
DataFrame Shape is (572424, 3)
Current Time is Fri Jan 24, 2020 07:21:41 for Done with 
=================================== Creating Artist Metadata DB ===================================
Process [{0}] took 1.6 minutes.
Saving Master Artist M

,Genre,Style,Collaboration
1000,"{'Electronic': 55, 'Hip Hop': 1, 'Non-Music': ...","{'Techno': 46, 'Electro': 17, 'Downtempo': 1, ...","{'Dave Clarke': 54, 'Bang The Future': 1, 'Mr...."
1000500,{'Electronic': 1},{'Progressive Trance': 1},{'Club Pulse': 1}
100200,{'Hip Hop': 11},"{'Conscious': 7, 'Pop Rap': 2, '': 3}","{'Dike': 11, 'ABS (2)': 2, 'Creutzfeld&Jakob':..."
1002000,{},{},{}
1005400,{'Rock': 5},"{'Alternative Rock': 1, 'Indie Rock': 1, 'Math...","{'Gapeseed': 5, 'Gerling': 1}"


## Artist Albums DB

In [14]:
from masterdb import getArtistAlbumsDB

In [21]:
discArtistAlbumsDB = getArtistAlbumsDB(disc, force=False)
discArtistAlbumsDB.head()

Current Time is Sun Jan 26, 2020 19:01:34 for 
=================================== Creating Artist Albums DB ===================================
Using previously created Artist Albums DB
Loading data from /Volumes/Music/Discog/db/MasterArtistAlbumsDB.p
  --> This file is 375.0MB.
Loading /Volumes/Music/Discog/db/MasterArtistAlbumsDB.p


,Albums
1000,"{'Albums': {'44815': 'Archive One', '44831': '..."
1000500,{'Miscellaneous': {'1178875': 'Peak Controller'}}
100200,"{'Albums': {'50511': 'PottpÃ¼ree'}, 'Singles &..."
1002000,{'Instruments & Performance': {'1002108': 'Are...
1002600,{'Vocals': {'1180905': 'Quiero'}}


In [20]:
discArtistAlbumsDB = getArtistAlbumsDB(disc)
discArtistAlbumsDB.head()

Current Time is Sun Jan 26, 2020 18:58:04 for 
=================================== Creating Artist Albums DB ===================================
Loading ArtistID Data
Loading data from /Volumes/Music/Discog/db/ArtistIDToAlbumNames.p
  --> This file is 190.8MB.
Loading /Volumes/Music/Discog/db/ArtistIDToAlbumNames.p
Creating Pandas DataFrame for 712628 Artists
	Shape --> (712628, 1)
DataFrame Shape is (712628, 1)
Saving Master Artist Albums DB File
  --> This file is 375.0MB.
Current Time is Sun Jan 26, 2020 19:00:52 for Done with 
=================================== Creating Artist Albums DB ===================================
Process [{0}] took 2.8 minutes.


,Albums
1000,"{'Albums': {'44815': 'Archive One', '44831': '..."
1000500,{'Miscellaneous': {'1178875': 'Peak Controller'}}
100200,"{'Albums': {'50511': 'PottpÃ¼ree'}, 'Singles &..."
1002000,{'Instruments & Performance': {'1002108': 'Are...
1002600,{'Vocals': {'1180905': 'Quiero'}}


## Albums DB

In [26]:
from masterdb import getAlbumDB

In [27]:
discAlbumDB = getAlbumDB(disc)
discAlbumDB.head()

Current Time is Wed Jan 22, 2020 21:10:30 for 
=================================== Creating Artist Album DB ===================================
Loading AlbumID Data
Loading data from /Volumes/Music/Discog/db/AlbumIDToName.p
  --> This file is 39.7MB.
Loading /Volumes/Music/Discog/db/AlbumIDToName.p
Loading data from /Volumes/Music/Discog/db/AlbumIDToRef.p
  --> This file is 55.9MB.
Loading /Volumes/Music/Discog/db/AlbumIDToRef.p
Loading data from /Volumes/Music/Discog/db/AlbumIDToArtists.p
  --> This file is 22.8MB.
Loading /Volumes/Music/Discog/db/AlbumIDToArtists.p
Creating Pandas DataFrame for 1695744 Albums
	Shape --> (1695744, 1)
  Joining Ref
	Shape --> (1695744, 2)
  Joining Artists
	Shape --> (1695744, 3)
DataFrame Shape is (1695744, 3)
Current Time is Wed Jan 22, 2020 21:11:41 for Done with 
=================================== Creating Artist Album DB ===================================
Process [{0}] took 1.2 minutes.


,Name,Ref,Artists
1012077,Live,/Dave-Clarke-Live/master/1012077,[1000]
1258261,Walls Of Genius And Miracle,/Walls-Of-Genius-and-Miracle-Walls-Of-Genius-A...,"[1009227, 182635, 1000]"
2546978,RA.EX007 Dave Clarke,/Dave-Clarke-RAEX007-Dave-Clarke/release/2546978,[1000]
44815,Take Two,/6th-Sense-Approach-Take-Two/release/44815,"[23951, 1000]"
44831,Devil's Advocate,/Dave-Clarke-Devils-Advocate/master/44831,[1000]


## Artist Album ID --> Known Albums

In [28]:
from masterdb import getArtistAlbumKnownDB

In [29]:
discArtistAlbumKnownDB = getArtistAlbumKnownDB(discAlbumDB, discArtistAlbumsDB)
discArtistAlbumKnownDB.head()

Current Time is Wed Jan 22, 2020 21:11:41 for 
=================================== Creating Artist Album DB ===================================
Creating Pandas DataFrame for 712336 Arist Albums
	Shape --> (712336, 3)
DataFrame Shape is (712336, 3)
Current Time is Wed Jan 22, 2020 21:12:04 for Done with 
=================================== Creating Artist Album DB ===================================
Process [{0}] took 23 seconds.


,Known Albums,All Albums,Albums
1000,53,53,"{'44815': ['Archive One', 'Albums', True], '44..."
1000500,1,1,"{'1178875': ['Peak Controller', 'Miscellaneous..."
100200,9,9,"{'50511': ['PottpÃ¼ree', 'Albums', True], '219..."
1002000,3,3,"{'1002108': ['Are You Faithful?', 'Instruments..."
1002600,1,1,"{'1180905': ['Quiero', 'Vocals', True]}"


# Joining Artist ID DataFrame

In [30]:
from masterdb import createMasterDB

In [31]:
createMasterDB(disc, discArtistDB, discArtistMetadataDB, discArtistAlbumKnownDB)

Current Time is Wed Jan 22, 2020 21:12:04 for 
=================================== Creating Artist ID DB ===================================
Creating Pandas DataFrame for 712336 Arist IDs
  Joining Artist Metadata
	Shape --> (712336, 9)
  Joining Artist Albums
	Shape --> (712336, 12)
Current Time is Wed Jan 22, 2020 21:12:10 for Done with 
=================================== Creating Artist ID DB ===================================
Process [{0}] took 5 seconds.
Saving data to /Volumes/Music/Discog/db/MasterDB.p
  --> This file is 623.0MB.
Saved data to /Volumes/Music/Discog/db/MasterDB.p
  --> This file is 623.0MB.


In [ ]:
from random import random